In [1]:
!pip install -q transformers
!pip install pyspellchecker==0.7.2

In [4]:
import re
import nltk
import spacy
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from scipy.sparse import csr_matrix
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from transformers import pipeline
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from Utils.MetricasModelos import ModelMetrics

In [5]:
class OrtographyCorrector:
    def __init__(self, language='es', dictionary_file='./Data/diccionario.txt'):
        self.spell = SpellChecker(language=language)
        self.dictionary = self.load_dictionary(dictionary_file)

    def load_dictionary(self, dictionary_file):
        with open(dictionary_file, 'r', encoding='utf-8') as file:
            return file.read().splitlines()

    def correct_sentence(self, sentence):
        words = sentence.split()
        corrected_words = [self.correct_word(word) for word in words]
        return ' '.join(corrected_words)

    def correct_word(self, word):
        if self.spell.correction(word) == word:
            return word
        else:
            suggestions = list(self.spell.candidates(word))
            matched_suggestions = [s for s in suggestions if s in self.dictionary]

            if matched_suggestions:
                corrected_word = random.choice(matched_suggestions)
            else:
                corrected_word = str(suggestions[0])

            return corrected_word

In [6]:
df = pd.read_csv('./Data/Sentiment_Data/twitter_training.csv',
                       header=None,
                       names=['TweetID', 'Sentiment', 'TweetText'],)

df_train = df.sample(n=1000, random_state=1)
df_train

,TweetID,Sentiment,TweetText
2415,Borderlands,Positive,FICK YESSSSSS.
8319,Microsoft,Negative,Why are people talking about microsoft buying ...
9181,Nvidia,Neutral,Nvidia's ready to announce the bad stuff.. Rea...
7841,MaddenNFL,Negative,Gee. I didn't realize Madden NFL needed QBs fo...
575,ApexLegends,Positive,The game is an absolute shredder!!!
...,...,...,...
5048,GrandTheftAuto(GTA),Positive,It's about fricking time!
9427,Overwatch,Negative,"Initially, I didn't like the idea, but I would..."
1653,CallOfDutyBlackopsColdWar,Positive,This video has only brought back good memories...
1675,CallOfDutyBlackopsColdWar,Neutral,The very best and the game isn’t out yet


In [7]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Tokenización
def tokenize_text(text):
    return word_tokenize(text)

# Lematización
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(text)])

def lemmatize_text_lang(text, language):
    nlp = spacy.load(f"{language}_core_news_sm")
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Modificación de la lista de stopwords
def modify_stopwords(text, new_stopwords=[], remove_stopwords=[]):
    stop_words = set(stopwords.words('spanish'))

    # Agrega nuevas stopwords
    for word in new_stopwords:
        stop_words.add(word)

    # Elimina stopwords
    for word in remove_stopwords:
        stop_words.discard(word)

    return ' '.join([word for word in word_tokenize(text) if word not in stop_words])

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [8]:
sentiment_analysis = pipeline(task="sentiment-analysis",
                              framework="pt",
                              model="j-hartmann/sentiment-roberta-large-english-3-classes")

Some weights of the model checkpoint at j-hartmann/sentiment-roberta-large-english-3-classes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [9]:
df_train['TweetText'] = df_train['TweetText'].astype(str)
df_train['Sentiment'] = df_train['Sentiment'].str.lower()
list_text = df_train['TweetText'].tolist()

In [10]:
results = sentiment_analysis(list_text)
df_train['HuggingFace_Sentiment'] = [result['label'] for result in results]

In [11]:
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_sentiment_label(score):
    if score > 0.05:
        label = 'positive'
    elif score < -0.05:
        label = 'negative'
    else:
        label = 'neutral'
    return label

df_train['nltk_sentiment'] = df_train['TweetText'].apply(
    lambda x: get_sentiment_label(sia.polarity_scores(x)['compound']))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/diego/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
# Normalizacion y limpieza
df_train['TweetText'] = df_train['TweetText'].str.lower()

def remove_symbols(text, regex):
    return re.sub(regex, '', text)

df_train['TweetText'] = df_train['TweetText'].apply(lambda x: remove_symbols(x, r'[^\w\s]'))
df_train['TweetText'] = df_train['TweetText'].apply(lambda x: remove_symbols(x, r'\d+'))
df_train['TweetText'] = df_train['TweetText'].str.lower()
print(df_train)

                        TweetID Sentiment  \
2415                Borderlands  positive   
8319                  Microsoft  negative   
9181                     Nvidia   neutral   
7841                  MaddenNFL  negative   
575                 ApexLegends  positive   
...                         ...       ...   
5048        GrandTheftAuto(GTA)  positive   
9427                  Overwatch  negative   
1653  CallOfDutyBlackopsColdWar  positive   
1675  CallOfDutyBlackopsColdWar   neutral   
3834              Cyberpunk2077  positive   

                                              TweetText HuggingFace_Sentiment  \
2415                                      fick yessssss              positive   
8319  why are people talking about microsoft buying ...              negative   
9181  nvidias ready to announce the bad stuff really...              negative   
7841  gee i didnt realize madden nfl needed qbs for ...              negative   
575                    the game is an absolute shredde

In [16]:
df_train['TweetText'] = df_train['TweetText'].astype(str)
list_text = df_train['TweetText'].tolist()
results = sentiment_analysis(list_text)
df_train['Preprocessing+HuggingFace_Sentiment'] = [result['label'] for result in results]

In [17]:
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_sentiment_label(score):
    if score > 0.05:
        label = 'positive'
    elif score < -0.05:
        label = 'negative'
    else:
        label = 'neutral'
    return label

df_train['Preprocessing+nltk_sentiment'] = df_train['TweetText'].apply(
    lambda x: get_sentiment_label(sia.polarity_scores(x)['compound']))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/diego/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [19]:
metricas = ModelMetrics()

df_metrics = {'Precision': [], 'Exactitud': [], 'Sensibilidad': [], 'Especificidad': []}
for pred in [df_train['HuggingFace_Sentiment'],
             df_train['nltk_sentiment'],
             df_train['Preprocessing+HuggingFace_Sentiment'],
             df_train['Preprocessing+nltk_sentiment']]:
    df_metrics['Precision'].append(metricas.precision(df_train['Sentiment'], pred))
    df_metrics['Exactitud'].append(metricas.accuracy(df_train['Sentiment'], pred))
    df_metrics['Sensibilidad'].append(metricas.recall(df_train['Sentiment'], pred))
    df_metrics['Especificidad'].append(metricas.specificity(df_train['Sentiment'], pred))
df_metrics = pd.DataFrame(df_metrics, index=['HuggingFace',
                                             'NLTK',
                                             'HuggingFace+Preprocessing',
                                             'NLTK+Preprocessing'])
df_metrics

 [20-Jun-23 12:20:35] [INFO] [ModelMetrics]  =>>>   Calculating precision
 [20-Jun-23 12:20:35] [INFO] [ModelMetrics]  =>>>   Calculating accuracy
 [20-Jun-23 12:20:35] [INFO] [ModelMetrics]  =>>>   Calculating recall
 [20-Jun-23 12:20:35] [INFO] [ModelMetrics]  =>>>   Calculating specificity
 [20-Jun-23 12:20:35] [INFO] [ModelMetrics]  =>>>   Calculating precision
 [20-Jun-23 12:20:35] [INFO] [ModelMetrics]  =>>>   Calculating accuracy
 [20-Jun-23 12:20:35] [INFO] [ModelMetrics]  =>>>   Calculating recall
 [20-Jun-23 12:20:36] [INFO] [ModelMetrics]  =>>>   Calculating specificity
 [20-Jun-23 12:20:36] [INFO] [ModelMetrics]  =>>>   Calculating precision
 [20-Jun-23 12:20:36] [INFO] [ModelMetrics]  =>>>   Calculating accuracy
 [20-Jun-23 12:20:36] [INFO] [ModelMetrics]  =>>>   Calculating recall
 [20-Jun-23 12:20:36] [INFO] [ModelMetrics]  =>>>   Calculating specificity
 [20-Jun-23 12:20:36] [INFO] [ModelMetrics]  =>>>   Calculating precision
 [20-Jun-23 12:20:36] [INFO] [ModelMetrics] 

,Precision,Exactitud,Sensibilidad,Especificidad
HuggingFace,0.512,0.512,0.512,0.833519
NLTK,0.405,0.405,0.405,0.793585
HuggingFace+Preprocessing,0.496,0.496,0.496,0.827797
NLTK+Preprocessing,0.409,0.409,0.409,0.794667


Para este caso en particular con la base de datos que se tiene, no se observa buenas metricas pero tambien hay que tener que las metricas que se estan generando son medias de las 3 etiquetas. Toca revisar a fondo cuales de las etiquetas logra identificar mejor, además se podria utilizar un corrector ortografico para ver el comportamiento tambien se puede realizar eliminacion de stopwords y otros procesos para poder mejorar los resultados.

**1. ¿Cuáles son los diferentes tipos de análisis de sentimientos?**

Existen varios tipos de análisis de sentimientos, incluyendo:

- **Análisis de sentimientos a nivel de polaridad:** Este tipo de análisis se enfoca en identificar la polaridad general de un texto, es decir, si es positivo, negativo o neutral.

- **Análisis de sentimientos a nivel de emoción:** Este tipo va más allá de la polaridad e intenta identificar emociones específicas, como alegría, tristeza, enojo, sorpresa, etc.

- **Análisis de sentimientos a nivel de aspecto:** Este tipo de análisis busca identificar sentimientos o emociones relacionados con aspectos específicos dentro de un texto. Por ejemplo, en una revisión de un producto, un usuario podría tener una opinión positiva sobre la duración de la batería, pero una opinión negativa sobre la interfaz de usuario.

- **Análisis de sentimientos a nivel de intención:** Este tipo de análisis se enfoca en identificar la intención del autor, como recomendar, no recomendar, comprar, no comprar, etc.

**2. ¿Cuáles son los diferentes métodos para el análisis de sentimientos?**

Los métodos para el análisis de sentimientos se pueden dividir en dos categorías generales:

- **Métodos basados en lexicones:** Estos métodos utilizan listas predefinidas de palabras asociadas con sentimientos positivos o negativos. El análisis de sentimientos se realiza contando la frecuencia de las palabras positivas y negativas y decidiendo la polaridad general del texto.

- **Métodos basados en aprendizaje automático:** Estos métodos utilizan técnicas de aprendizaje automático (como la regresión logística, máquinas de vectores de soporte, redes neuronales, etc.) para aprender a partir de un conjunto de datos de entrenamiento etiquetado y luego aplicar este aprendizaje para predecir el sentimiento de los datos no etiquetados.

**3. ¿Cuáles son los desafíos del análisis de sentimientos?**

El análisis de sentimientos presenta varios desafíos:

- **Sarcasmo e ironía:** Estos pueden ser particularmente difíciles de detectar para los algoritmos de análisis de sentimientos, ya que a menudo implican el uso de palabras positivas para expresar sentimientos negativos y viceversa.

- **Ambigüedad:** La ambigüedad lingüística puede dificultar la determinación de la polaridad de un texto. Una palabra puede tener diferentes connotaciones dependiendo del contexto en el que se use.

- **Aspectos múltiples:** Como se mencionó anteriormente, un texto puede contener sentimientos diferentes hacia diferentes aspectos. Esto puede dificultar la determinación de un sentimiento general.

- **Disponibilidad de datos etiquetados:** Para los métodos de aprendizaje automático, se necesita un conjunto de datos de entrenamiento grande y de alta calidad con etiquetas de sentimientos. Estos pueden ser difíciles y costosos de obtener. 

- **Aspectos culturales y de lenguaje:** Los sentimientos pueden expresarse de manera diferente en diferentes culturas o en diferentes idiomas. Esto puede hacer que el análisis de sentimientos sea más complejo.